## Setup libraries

In [40]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [41]:
SchivenogliaAir = read_csv("AirQualitySchivenoglia14-20.csv")
#Schivenoglia = Schivenoglia.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                       "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

SchivenogliaAir = SchivenogliaAir.drop(["Benzene", "Arsenic", "Lead", "Nikel", "Benzo_a_pyrene", "Cadmium"], axis='columns')

print(SchivenogliaAir.shape)
SchivenogliaAir.head()



(2552, 11)


,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,0.716667,28.733333,53.554167,12.645833,56.0,56.0,2.887500
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,0.790909,32.304762,67.661905,4.640909,56.0,55.0,3.522727
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,0.612500,33.179167,57.995833,5.395833,43.0,39.0,2.154167
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,0.708333,28.166667,56.179167,6.750000,37.0,33.0,3.108333
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,0.737500,28.325000,65.870833,3.720833,18.0,16.0,4.116667


In [42]:
SchivenogliaAdd= read_csv("RisultatiSchivenoglia.csv")
#Cremona = Cremona.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                       "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

SchivenogliaAdd = SchivenogliaAdd.drop(["Benzene","Benzo_a_pyrene", "Cadmium","Lead", "Nikel","Unnamed: 0", "Arsenic","NameStation.1", "NameStation", "IDStation", "IDStation.1", "Date.1", "PM10", "Ammonia", "PM25"], axis='columns')
print(SchivenogliaAdd.shape)
SchivenogliaAdd.rename(columns={'Date': 'Date1'}, inplace=True)
SchivenogliaAdd.head()

(2524, 7)


,Date1,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,2014-01-01,1.508213,260.0,4.766667,84.158333,79.737500,0.2
1,2014-01-02,0.390941,332.0,5.479167,87.091667,13.875000,2.0
2,2014-01-03,2.104680,267.0,5.766667,97.412500,23.129167,1.4
3,2014-01-04,0.949083,104.0,6.950000,99.608333,4.858333,5.8
4,2014-01-05,1.130128,248.0,7.541667,99.829167,5.970833,14.2


## Joining dataframe

In [43]:
Schivenoglia = sqldf('''
SELECT  * 
FROM SchivenogliaAir CA JOIN SchivenogliaAdd CD
ON CA.Date = CD.Date1
''')

Schivenoglia= Schivenoglia.drop(["Date1"], axis='columns')

print(Schivenoglia.shape)
Schivenoglia.head()

(2524, 17)


,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,0.716667,28.733333,53.554167,12.645833,56.0,56.0,2.887500,1.508213,260.0,4.766667,84.158333,79.737500,0.2
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,0.790909,32.304762,67.661905,4.640909,56.0,55.0,3.522727,0.390941,332.0,5.479167,87.091667,13.875000,2.0
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,0.612500,33.179167,57.995833,5.395833,43.0,39.0,2.154167,2.104680,267.0,5.766667,97.412500,23.129167,1.4
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,0.708333,28.166667,56.179167,6.750000,37.0,33.0,3.108333,0.949083,104.0,6.950000,99.608333,4.858333,5.8
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,0.737500,28.325000,65.870833,3.720833,18.0,16.0,4.116667,1.130128,248.0,7.541667,99.829167,5.970833,14.2


# Quadrants elaboration

In [44]:
#Primo quadrante
Q1 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 0) &
                 (Schivenoglia.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "IDStation","NameStation","Date", "Ammonia", "CO", "NO2", "NOx","Ozone", "PM10", "PM25", "Sulfur_dioxide",
                 "Wind_speed",  "Temperature","Relative_humidity","Global_radiation", "Rainfall", "Wind_direction" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 90) &
                 (Schivenoglia.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 180) &
                 (Schivenoglia.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 270) &
                 (Schivenoglia.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Schivenoglia = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Cremona['Wind_direction']
Schivenoglia.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Schivenoglia

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4173/213750803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4173/213750803.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4173/213750803.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Temperature,Relative_humidity,Global_radiation,Rainfall,Quadrante
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,0.716667,28.733333,53.554167,12.645833,56.0,56.0,2.887500,1.508213,4.766667,84.158333,79.737500,0.2,3
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,0.790909,32.304762,67.661905,4.640909,56.0,55.0,3.522727,0.390941,5.479167,87.091667,13.875000,2.0,4
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,0.612500,33.179167,57.995833,5.395833,43.0,39.0,2.154167,2.104680,5.766667,97.412500,23.129167,1.4,3
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,0.708333,28.166667,56.179167,6.750000,37.0,33.0,3.108333,0.949083,6.950000,99.608333,4.858333,5.8,2
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,0.737500,28.325000,65.870833,3.720833,18.0,16.0,4.116667,1.130128,7.541667,99.829167,5.970833,14.2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,703,Schivenoglia Via Malpasso,2020-12-27,NaN,0.882609,14.026087,16.886957,32.760870,23.0,21.0,4.030435,0.519307,1.945833,84.305556,81.204861,0.0,4
2473,703,Schivenoglia Via Malpasso,2020-12-28,NaN,0.887500,18.212500,20.308333,34.141667,18.0,14.0,6.762500,1.756182,1.222222,99.542361,8.741667,7.0,4
2474,703,Schivenoglia Via Malpasso,2020-12-29,5.0,0.991667,27.850000,37.066667,17.662500,33.0,32.0,6.470833,1.019866,2.134722,99.568056,25.305556,0.4,1
2475,703,Schivenoglia Via Malpasso,2020-12-30,5.9,1.126087,40.120833,51.200000,4.995833,52.0,48.0,3.545833,2.195260,2.540278,99.589583,18.256944,0.0,3


In [45]:
Schivenoglia

,IDStation,NameStation,Date,Ammonia,CO,NO2,NOx,Ozone,PM10,PM25,Sulfur_dioxide,Wind_speed,Temperature,Relative_humidity,Global_radiation,Rainfall,Quadrante
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,0.716667,28.733333,53.554167,12.645833,56.0,56.0,2.887500,1.508213,4.766667,84.158333,79.737500,0.2,3
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,0.790909,32.304762,67.661905,4.640909,56.0,55.0,3.522727,0.390941,5.479167,87.091667,13.875000,2.0,4
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,0.612500,33.179167,57.995833,5.395833,43.0,39.0,2.154167,2.104680,5.766667,97.412500,23.129167,1.4,3
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,0.708333,28.166667,56.179167,6.750000,37.0,33.0,3.108333,0.949083,6.950000,99.608333,4.858333,5.8,2
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,0.737500,28.325000,65.870833,3.720833,18.0,16.0,4.116667,1.130128,7.541667,99.829167,5.970833,14.2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,703,Schivenoglia Via Malpasso,2020-12-27,NaN,0.882609,14.026087,16.886957,32.760870,23.0,21.0,4.030435,0.519307,1.945833,84.305556,81.204861,0.0,4
2473,703,Schivenoglia Via Malpasso,2020-12-28,NaN,0.887500,18.212500,20.308333,34.141667,18.0,14.0,6.762500,1.756182,1.222222,99.542361,8.741667,7.0,4
2474,703,Schivenoglia Via Malpasso,2020-12-29,5.0,0.991667,27.850000,37.066667,17.662500,33.0,32.0,6.470833,1.019866,2.134722,99.568056,25.305556,0.4,1
2475,703,Schivenoglia Via Malpasso,2020-12-30,5.9,1.126087,40.120833,51.200000,4.995833,52.0,48.0,3.545833,2.195260,2.540278,99.589583,18.256944,0.0,3


# Exporting data

In [46]:
Schivenoglia.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Schivenoglia/QuadrantiNuovo/QuadrantiSchivenoglia.csv')

